<a href="https://colab.research.google.com/github/Dahuginn/4yr_animerecommend/blob/main/%5BThesis%5Dgraph_split_node2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install node2vec
!pip install stellargraph

In [2]:
import pandas as pd
import numpy as np

import networkx as nx
import stellargraph as sg
from sklearn.model_selection import train_test_split
from stellargraph.data import EdgeSplitter
from sklearn.manifold import TSNE

from node2vec import Node2Vec

from google.colab import drive
drive.mount('/content/gdrive')
import sys
sys.path.append('/content/gdrive/My Drive')
import pickle

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
fh = open("/content/gdrive/My Drive/4 year/graph.graphml", "rb")
graph = nx.read_graphml(fh)
nx.info(graph)

'Graph with 2972 nodes and 880236 edges'

In [4]:
edge_splitter_test = EdgeSplitter(graph)
graph_test, examples_test, labels_test = edge_splitter_test.train_test_split(p=0.01, method="global")

print(nx.info(graph_test))

** Sampled 8802 positive and 8802 negative edges. **
Graph with 2972 nodes and 871434 edges


In [5]:
edge_splitter_train = EdgeSplitter(graph_test, graph)
graph_train, examples, labels = edge_splitter_train.train_test_split(p=0.01, method="global")
examples_train, examples_model_selection, labels_train, labels_model_selection = train_test_split(examples, labels, train_size=0.75, test_size=0.25)

print(nx.info(graph_train))

** Sampled 8714 positive and 8714 negative edges. **
Graph with 2972 nodes and 862720 edges


In [6]:
pd.DataFrame(
    [("Training", len(examples_train), "Train classifier"),
     ("Model selection", len(examples_model_selection), "Choose the best model"),
     ("Test", len(examples_test), "Determine the best classifier score")],
     columns=("Split", "Num ", "Purpose")).set_index("Split")

,Num,Purpose
Split,,
Training,13071,Train classifier
Model selection,4357,Choose the best model
Test,17604,Determine the best classifier score


In [ ]:
node2vec = Node2Vec(graph_train, dimensions=7, walk_length=3, num_walks=3, weight_key='weight')
model = node2vec.fit(window=3, min_count=1)

Computing transition probabilities:   0%|          | 0/2972 [00:00<?, ?it/s]

In [ ]:
pickle.dump(model, open('/content/gdrive/My Drive/4 year/node2vec.sav', 'wb'))

In [ ]:
loaded_model = pickle.load(open('/content/gdrive/My Drive/4 year/node2vec.sav', 'rb'))

In [ ]:
ne = model.wv.vectors
tsne2 = TSNE(n_components=2, random_state=42)
ne_2d = tsne2.fit_transform(ne)

In [ ]:
plt.figure(figsize=(10, 8))
plt.scatter(ne_2d[:, 0], ne_2d[:, 1])
plt.show()

In [ ]:
print(names_available[306], model.wv.get_vector(names_available[306]))

In [ ]:
for node, score in model.wv.most_similar('Sonic X TV Adventure, Comedy, Kids, Mecha, Sci-Fi, Shounen'):
  print(node, score)

In [ ]:
arr = []
for i in examples_train_nx[:5]:
  a, b = i
  arr.append(model.wv.get_vector(a))
  arr.append(model.wv.get_vector(b))

In [ ]:
data_tuples = list(zip(examples_train_nx, labels_train_nx))

In [ ]:
df = pd.DataFrame(data_tuples, columns=['features', 'labels'])

In [ ]:
features = []
for i in df.features:
  a, b = i
  features.append(a + ' ' + b)

In [ ]:
df.features = features
df

In [ ]:
for node, score in model.wv.most_similar('Sonic X TV Adventure, Comedy, Kids, Mecha, Sci-Fi, Shounen'):
  print(node, score)